# Abstract

토픽모델이 문서를 특정 주제들로 표현한다는 해석적 장점이 있는 반면 단어의 분산 벡터 표현인(**Distribution dense word vectors**) 워드임베딩은 토큰 간의 의미적 유사성을 포착한다는 장점이 있다.

그러나 토픽 모델의 경우 문서에서 단어의 출현 빈도수에 기반한 방법이기 때문에 의미적 유사성을 포착하지 못하고 워드 임베딩의 경우 개별 단어가 가지는 벡터값이 어떤 의미를 표현하고 있는지 직관적으로 알기 어렵다는 단점이 있다.

본 논문에서는 LDA와 워드임베딩을 결합한 상호보완적인 기법인 **Lda2vec**을 소개한다. 

# Introduction

토픽 모델링은 다양한 분야에서 문서 집합을 중요한 주제들로 조직화하는 기법으로 사용된다. 워드임베딩과 다르게 해당 문서가 어떠한 주제를 다루는지 쉽게 파악할 수 있도록 해석적 측면에서 강점을 지닌다.

또한 이러한 장점은 문서집합에서 눈에 띄는 패턴과 구조를 쉽게 파악할 수 있도록 해준다. 수집된 문서들의 단어들을 기반으로 군집화, 트렌드 파악, 상품간 네트워크를 파악하는데 사용된 사례들이 존재하며 더 나아가 컴퓨터 비전, 사회 연결망 구축, 이미지 분석 등에서도 토픽 모델링이 사용되고 있다.

그러나 이러한 전통적인 토픽 모델들은 문서내 단어 출현 빈도의 기반한 기법이기 때문의 단어간 의미적 유사성을 고려하지 못한다. 반면에 워드임베딩의 경우 설정한 범위 내 주변단어과 관계를 학습해 나가며 단어간 의미적 유사성을 파악할 수 있다. 이것을 수식화하면 다음과 같다.

> example :  `king` -`man` + `women` = `queen`

본 논문에서는 위와 같은 워드 임베딩 기법중에서 $\mathrm{Skipgram\,\, Negative\,\, Sampling\,(SGNS)}$을 사용해 기존의 해석적 장점을 가진 $\mathrm{LDA}$ 모델과 결합하여 토픽 형성시 단어간 의미적 유사성을 반영할 수 있도록 한다. 

# Model

![](loss.png)

## Word Representation

$$L = L^d + \sum_{ij}L_{ij}^{neg}$$

$$L^{neg}_{ij} = \log \sigma\,(\vec{c}_{j}\cdot \vec{w_i}) + \sum_{l=0}^n \log \sigma\,(-\vec{c}_{j}\cdot \vec{w_l})$$

* $L : \mathrm {total\,loss\,term}$

* $L^{neg}_{ij} : \mathrm {Skipgram \, Negative\, Sampling \, Loss}$

* $\vec {c_j} = \vec{w_j} + \vec{d_j}$ : $\mathrm {context \, vector}$

* $\vec{w_j} : \mathrm {pivot\,\,word\,\,vector}$, $\vec{w_i} : \mathrm {target\,\,word\,\,vector}$, $\vec{w_l} : \mathrm {nagative-sampled\,word\,vector}$

위 수식은 본 논문에서 사용된 손실함수이다. 본 논문에서는 윈도우 사이즈를 5로 설정하였고 네거티브 샘플링에 사용될 샘플 수는 15($=n$)으로 설정하였다.

## Document Representation

lda2vec은 단어 벡터와 문서 벡터를 같은 공간에 표현하고 동시에 학습한다. 다음과 같은 예시를 들어보자

예를 들어 `한국` 이라는 단어가 주어졌다고 하자. 워드임베딩 관점에서 생각해보면 `한국`이라는 단어가 등장했을 때 `김치`, `K-pop`, `한복` 이라는 연관 단어들을 생각해 볼 수 있다.

그런데 여기서 해당 문서가 `음식`과 관련된 문서라는 정보가 추가되었다고 하자. 그렇다면 `한국` 이라는 단어가 등장했을 때 구체적으로 `김치`, `불고기`. `배달의 민족` 등등의 단어들을 연상할 수 있는 것이다.

즉, lda2vec가 의도한 바는 토픽 형성시 이러한 토픽 정보와 단어 정보를 결합한 후 학습해 나가면서 단어의 의미적 유사성과 해석적 측면을 모두 반영하여 토픽을 형성하자는 것이다.

### Document Mixtures

![](docvector.png)

$$\vec{d_j} = p_{j0}\cdot\vec{t_0} + p_{j1}\cdot\vec{t_1} \dots  + p_{jn}\cdot\vec{t_n}$$

$$ 0 \leq p_{jk} \leq 1, \quad \vec{t_k} : \mathrm{topic \,vector}$$

* $p_{jk}$ : $j$번째 문서의 $k$번째 토픽 비율

* $\vec{t_k}$ : $k$번째 토픽 벡터

### Sparse Membership

![](ld.png)

$$L = L^d + \sum_{ij}L_{ij}^{neg}$$

$$L^{neg}_{ij} = \log \sigma\,(\vec{c}_{j}\cdot \vec{w_i}) + \sum_{l=0}^n \log \sigma\,(-\vec{c}_{j}\cdot \vec{w_l})$$ㅡ

$$L^d = \lambda \sum_{jk} (\alpha-1)\log p_{jk}$$

$$ 0 \leq p_{jk} \leq 1$$

$$\lambda,\alpha : \mathrm{tuning\, parameter}$$  

* $L^d $ :   $\mathrm{Dirichlet\,likelihood\,loss}$

* $\lambda$ : $\mathrm{total\, loss}$ 계산시

* $\alpha : $ 희소성 유도 파라미터

$\alpha<1$ 인 경우 sparse한 성질 즉 특정 토픽으로 수렴할 정도가 높아지고  반대의 경우 문서에 토픽확률들이 동일하게 될 가능성이 높아진다.(로그함수 그래프를 생각하면 쉽다.)

본 논문에서는 토픽 수인 $n= 20$으로 설정하여 $\alpha = n^{-1}$로 설정하였으며, 옴티마이저함수로 Adam, 200epochs를 수행하여 학습하였다.